In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# Define the transform to convert images to tensors and normalize them
# transforms.ToTensor scales the input into [0,1]
# then the normalize takes mean and std values per channel and applies 
# (x - mean) / std so we get arrays with values in [-1, 1] by taking mean=0.5
# and std = 0.5
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])


# Download and load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [2]:
a, e = trainset[0]
print(a.shape)
input_feat_no = a.shape[-2]* a.shape[-1]

torch.Size([1, 28, 28])


In [3]:
# Define the neural network architecture
class Net(torch.nn.Module):
    def __init__(self, layer_sizes):
        super(Net, self).__init__()
        input, h1, h2, out = layer_sizes
        self.fc1 = torch.nn.Linear(input, h1)
        self.fc2 = torch.nn.Linear(h1, h2)
        self.fc3 = torch.nn.Linear(h2, out)

    def forward(self, x):
        x = x.view(-1, x.shape[-2] * x.shape[-1])
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
layer_sizes = [input_feat_no, 256, 128, 10]

In [6]:
# create network object
net = Net(layer_sizes)

# Define the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
epoch_num = 2
steps_for_loss = 200
# Train the neural network
for epoch in range(epoch_num):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % steps_for_loss == steps_for_loss - 1:    # print every steps_for_loss mini-batches
            print('epoch={}, step={} loss: {}'.format(epoch+1, i+1, running_loss / steps_for_loss))
            running_loss = 0.0

print('Finished Training')

epoch=1, step=200 loss: 2.0477713590860365
epoch=1, step=400 loss: 1.157635293006897
epoch=1, step=600 loss: 0.6457270790636539
epoch=1, step=800 loss: 0.4961037540435791
epoch=2, step=200 loss: 0.4057026132941246
epoch=2, step=400 loss: 0.38611757136881353
epoch=2, step=600 loss: 0.36422798424959185
epoch=2, step=800 loss: 0.34328299164772036
Finished Training


In [8]:
# Test the neural network on test data
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}')

Accuracy of the network on the 10000 test images: 90.47
